In [1]:
!pip install psycopg2-binary pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 32.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 52.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 8.2 MB/s eta 0:00:00:00:01


In [2]:
topics = []
with open("./inputs/topics.txt", "r") as file:
    topics = file.read().split()
topics

['Sports-and-Outdoors',
 'Home-and-Kitchen',
 'Grocery-and-Gourmet-Food',
 'Automotive',
 'Software',
 'Office-Products',
 'Clothing-Shoes-and-Jewelry',
 'Baby',
 'Cell-Phones-and-Accessories',
 'Patio-Lawn-and-Garden',
 'Beauty',
 'Musical-Instruments',
 'Health-and-Personal-Care',
 'Pet-Supplies',
 'Electronics',
 'Tools-and-Home-Improvement',
 'Toys-and-Games',
 'Industrial-and-Scientific',
 'Video-Games',
 'Arts-Crafts-and-Sewing',
 'Appliances']

In [3]:
from collections import defaultdict
from psycopg2 import sql
import psycopg2
import pandas as pd

In [4]:
def get_column_names(conn, table):
    """
    Fetches and prints the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

In [6]:
# QUERY ON ALL TOPIC NODES
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

for topic in topics:
    conn = psycopg2.connect(
                dbname=dbname,
                user=username,
                password=password,
                host=topic,
            )

    # Prepare and execute the query
    query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            count_topic[topic] = len(rows)
    except psycopg2.errors.UndefinedTable:  # Handle table not found error
        print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
        count_topic[topic] = 0  # Or handle the case as you see fit
    finally:
        conn.close()

# sort by desending frequency
count_topic = dict(sorted(count_topic.items(), key=lambda x: x[1], reverse=True))

print('Num Topic:', len(count_topic))
total = sum(count_topic.values())

print(count_topic)
print(total)


Num Topic: 21
{'Electronics': 314263, 'Home-and-Kitchen': 184439, 'Sports-and-Outdoors': 146891, 'Tools-and-Home-Improvement': 101088, 'Automotive': 89923, 'Cell-Phones-and-Accessories': 85865, 'Health-and-Personal-Care': 80496, 'Patio-Lawn-and-Garden': 59595, 'Toys-and-Games': 51486, 'Office-Products': 43608, 'Beauty': 42422, 'Pet-Supplies': 36607, 'Baby': 28933, 'Musical-Instruments': 23322, 'Clothing-Shoes-and-Jewelry': 22068, 'Arts-Crafts-and-Sewing': 21262, 'Grocery-and-Gourmet-Food': 19538, 'Video-Games': 13307, 'Industrial-and-Scientific': 12136, 'Software': 10636, 'Appliances': 9011}


1396896

In [17]:
# CUSTOM QUERY ON SINGLE NODE
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query
count_topic = defaultdict(int)

# test for topic[0]
topic = 'Home-and-Kitchen'
conn = psycopg2.connect(
        dbname=dbname,
        user=username,
        password=password,
        host=topic,
    )

query = sql.SQL("SELECT id, question, answer, keywords FROM {}").format(sql.Identifier(table))

try:
    with conn.cursor() as cur:
        header = get_column_names(conn, table)
        cur.execute(query)
        rows = cur.fetchall()
except psycopg2.errors.UndefinedTable:  # Handle table not found error
    print(f"Table '{table}' does not exist in database '{dbname}' on node '{topic}'")
finally:
    conn.close()

node_df = pd.DataFrame(rows, columns=header[:-1])
# node_df.sort_values('question', inplace=True)
node_df.head(10)

for idx, row in node_df[10400:10600].iterrows():
    print(row['id'])

# Display rows with null 'id'
# null_id_rows = node_df[pd.isnull(node_df['id'])]
# print(null_id_rows)

11840
11841
11842
11843
11844
11845
11846
11847
11848
11849
11850
11851
11852
11853
11854
11855
11856
11857
11858
11859
11860
11861
11862
11863
11864
11865
11866
11867
11868
11869
11896
11870
11871
11872
11873
11874
11875
11876
11877
11878
11879
11880
11881
11882
11883
11884
11885
11886
11887
11888
11889
11890
11891
11892
11893
11894
11895
11899
11900
11901
11902
11903
11904
11905
11906
11907
11908
11909
11910
11911
11912
11913
11914
11915
11916
11917
11918
11919
11920
11921
11922
11923
11924
11925
11926
11927
11928
11929
11930
11931
11932
11933
11934
11935
11936
11937
11938
11939
11940
11941
11942
11943
11944
11945
11946
11947
11948
11949
11950
11951
11952
11953
11954
11955
11956
11957
11958
11959
11960
11961
11962
11963
11964
11965
11966
11967
11969
11970
11971
11972
11973
11974
11975
11976
11977
11978
11979
11980
11981
11982
11983
11984
11985
11986
11987
11988
11989
11990
11991
11992
11993
13198
11994
11995
11996
11997
11998
11999
12000
12001
12002
12003
12004
12005
12006
12007
1200

In [9]:
input_df = pd.read_csv('Automotive.csv')
# input_df.drop('index', axis=1, inplace=True)
# input_df.sort_values('question', inplace=True)

# check if any id is null
input_df.head(10)

,question,answer,topic,index
0,What is the most useful length to get?,at least 20 feet.......heres why....say you ha...,Automotive,30273
1,Are these cables made of copper or aluminum?,Coleman's website does indeed say copper equiv...,Automotive,30274
2,I bought the Red Extra Heavy Duty. Is that too...,"For jumper cables, you can never have ""too muc...",Automotive,30275
3,"Hi, Being 20ft 4gauge how heavy is this?",Not nearly heavy enough. I keep them under my ...,Automotive,30276
4,Do these cables come with a bag?,No,Automotive,30277
5,Are the wires paired together? Am surprised bo...,"Yes, it's a twined cable. And why does it surp...",Automotive,30278
6,How many amps can this handle?,Per Coleman Cable specifications 4 gauge 20 fo...,Automotive,30279
7,Can I use this cables to boost a school bus ?,I would think so. I bought them to charge a pu...,Automotive,30280
8,What is the most useful length to get?,at least 20 feet.......heres why....say you ha...,Automotive,30281
9,Are these cables made of copper or aluminum?,Coleman's website does indeed say copper equiv...,Automotive,30282


In [9]:
node_df.dtypes
input_df.dtypes

id           int64
question    object
answer      object
keywords    object
dtype: object

In [23]:
def escape(s):
    result = ""
    for c in s:
        if c == "'":
            result += "''"  # Escape single quote with another single quote
        elif c == '"':
            result += '\\"'  # Escape double quote
        elif c == '\\':
            result += '\\\\'  # Escape backslash
        else:
            result += c
    return result

In [10]:
diff = []
for (index1, row1), (index2, row2) in zip(node_df.iterrows(), input_df.iterrows()):
    r1 = row1['question'].strip()
    r2 = row2['question'].strip()
    if r1 != r2:
        print(r1, '\n\n', r2)
        break
        diff.append(index1)
len(diff)

How is this motor different than the one Amazon sells that is called Lomanco Power Vent Attic Fan Motor 1/10hp 1100 RPM 115 Volts # F0510B2497 ? 

 How is this motor different than the one Amazon sells that is called "Lomanco Power Vent Attic Fan Motor 1/10hp 1100 RPM 115 Volts # F0510B2497 ?


0